## Writing Custom Java Loop

Sometimes it is favorable to use NeuraLogic on a lower level of abstraction and handle the learning loop manually rather than delegating this responsibility to one of the pre-built evaluators.

NeuraLogic, especially for the Java Backend, provides multiple ways to handle the learning loop, with different optimizations and levels of control over the loop.


### Simple problem

For the purposes of this tutorial, we will use the following simple problem that describes learning XOR. 
In contrast to writing loops for different backends, for the Java backend it is important what is the optimizer, learning rate and loss function set in the settings, as related operations are not managed by users in the loop.


In [19]:
from neuralogic.core.settings import Settings, Optimizer
from neuralogic.utils.data import XOR_Vectorized

settings = Settings(optimizer=Optimizer.SGD)


### Getting the NeuraLogic Layer
The first step to write custom loop is to get the NeuraLogic that serves for doing the forward propagation of our problem.

This can be done by either importing directly the `NeuraLogicLayer` from its module or by using `get_neuralogic_layer`.


In [20]:
from neuralogic.nn.java import NeuraLogicLayer

# or

from neuralogic.nn import get_neuralogic_layer
from neuralogic.core import Backend

NeuraLogicLayer = get_neuralogic_layer(Backend.JAVA)

To be able to learn our problem, it is required to built it first. Building the problem will do grounding behind the scenes and will process each query and example and yield computation graphs/networks for them as well as learnable parameters (model).

It is necessary to specify the backend, because each backend has different needs for the data representation.


In [21]:
model, dataset = XOR_Vectorized(Backend.JAVA, settings)

The next step is to initialize the `NeuraLogicLayer` which will provide us with the interface to do the learning.


In [22]:
layer = NeuraLogicLayer(model)

### Classic learning loop
The NeuraLogic's learning loop with the Java backend is not a lot different from loops that can be seen in different frameworks.

In [24]:
epochs = 1  # Specify the number of epochs

# Set our model to the training mode
# This step is not important for the following use case and can be omitted
# Training mode can be also specified using layer(..., train=True) 

layer.train()

for _ in range(epochs):
    # Loop over all samples in our dataset
    # Samples corresponds to examples and queries "zipped" together
    for sample in dataset.samples:  
        # Do the forward propagation for the sample
        loss = layer(sample)
        
        print(f"Output: {loss.output()}, Target: {loss.target()}, Loss: {loss.value()}")

        # Do backpropagation
        loss.backward()

Output: 0.0, Target: 0.0, Loss: 0.0
Output: 0.6887123076159376, Target: 1.0, Loss: 0.09690002742979466
Output: 0.5538237487509226, Target: 1.0, Loss: 0.19907324717867989
Output: 0.8375907129066443, Target: 0.0, Loss: 0.7015582023474607


### Optimizations of the learning loop

The approach described above is correct and offers more control over the learning, but it has some tradeoffs such as overhead as it is required to do multiple calls to Java, which can be expensive.

The NeuraLogic framework offers multiple options to deal with scenarios where it is beneficial to gain speed at the cost of loss of the control.


#### Auto backpropagation

In [25]:
epochs = 1

# In this case, it is important to set the training mode
# because otherwise the backpropagation will not be executed
layer.train()

for _ in range(epochs):
    for sample in dataset.samples:
        
        # Do the forward propagation and also the back propagation using auto_backprop argument
        error, _ = layer(sample, auto_backprop=True)
        print(f"Loss: {error}")

Loss: 0.0
Loss: 0.09386538738920944
Loss: 0.17754162880852017
Loss: 0.7099790723897439


Using `auto_backprop` argument in the forward propagation will also do backpropagation without the need of an additional call to the Java backend.

#### Samples batching

Another way to optimize the learning loop is to pass all samples to do the forward propagation at once. In this case, the backpropagation will be done without the requirement of the `auto_backprop` argument. The cost of this approach is the loss of some information such as the loss of specific sample.

In [26]:
epochs = 1

layer.train()  # Again, it is important to set the training mode for the same reasons as above

for _ in range(epochs):
    # Passing all samples
    # Sum of loss/error of all samples will be returned, as well as the number of samples
    total_error, num_of_samples = layer(dataset.samples)
    
    print(f"Average loss: {total_error / num_of_samples}")


Average loss: 0.2424097116470788


This approach can be optimized even more by delegating the looping over each epoch to the Java backend. This way, we will only get the information about the total loss and the number of samples in the last epoch.

In [27]:
epochs = 3

layer.train()  # Again, it is important to set the training mode for the same reasons as above

# Passing all samples and number of epochs to do
total_error, num_of_samples = layer(dataset.samples, epochs=epochs)

print(f"Average loss of the last epoch {total_error / num_of_samples}")

Average loss of the last epoch 0.23680661462663516


It is possible to set samples on the layer using `set_training_samples`, which will result in storing those samples on the Java side. Then we can do the forward propagation without any argument and thus without transferring all samples on each epoch to the Java.


In [28]:
epochs = 3

layer.train()  # Again, it is important to set the training mode for the same reasons as above
layer.set_training_samples(dataset.samples)  # Set training samples 

for _ in range(epochs):
    total_error, num_of_samples = layer()  # No arguments
    print(f"Average loss: {total_error / num_of_samples}")


Average loss: 0.2354814002980244
Average loss: 0.23429548825065938
Average loss: 0.23320663870237882


### Benchmark

> Following benchmarks serves as indicators of impacts of different optimizations. It is important to note that not all approaches offer the same level of control and our test problem (vectorized xor) is small. On different (larger) problems, the efficiency might differ more significantly.

In [29]:
import time
from neuralogic import initialize
from neuralogic.utils.data import Mutagenesis

epochs = 100


def prepare_learning():
    model, dataset = Mutagenesis(Backend.JAVA, settings)
    layer = NeuraLogicLayer(model)
    
    return dataset, layer


def benchmark_learning(fun):
    tests = 1
    total_time = 0
    
    for _ in range(tests):
        dataset, layer = prepare_learning()
        layer.train()
        
        start_time = time.perf_counter()
        fun(dataset, layer)
        total_time += time.perf_counter() - start_time
    
    return (total_time / tests) / epochs

In [30]:
def classic_learning(dataset, layer):
    for _ in range(epochs):
        for sample in dataset.samples:
            loss = layer(sample)
            loss.backward()

def classic_auto_backprop_learning(dataset, layer):
    for _ in range(epochs):
        for sample in dataset.samples:
            layer(sample, auto_backprop=True)

def sample_batching_learning(dataset, layer):
    for _ in range(epochs):
        layer(dataset.samples)

def sample_batching_learning_delegate_epochs(dataset, layer):
    layer(dataset.samples, epochs=epochs)

def sample_batching_learning_set_training_samples(dataset, layer):
    layer.set_training_samples(dataset.samples)
    
    for _ in range(epochs):
        layer()

#### Benchmark Results

> Can differ a lot

In [40]:
print("Classic learning\t\t\t", benchmark_learning(classic_learning))

Classic learning			 0.08290630794974277


In [41]:
print("Classic learning (auto_backprop=True)\t", benchmark_learning(classic_auto_backprop_learning))

Classic learning (auto_backprop=True)	 0.07494904701015911


In [42]:
print("Batched learning\t\t\t", benchmark_learning(sample_batching_learning))

Batched learning			 0.05744525757007068


In [45]:
print("Batched learning (epochs=...)\t\t", benchmark_learning(sample_batching_learning_delegate_epochs))

Batched learning (epochs=...)		 0.05674471384001663


In [46]:
print("Batched learning (set_training_samples)\t", benchmark_learning(sample_batching_learning_set_training_samples))

Batched learning (set_training_samples)	 0.05701841869013151


### Learning test

In [49]:
def prepare_xor_learning():
    model, dataset = XOR_Vectorized(Backend.JAVA, settings)
    layer = NeuraLogicLayer(model)
    
    return dataset, layer


def evaluate_testing(fun):
    dataset, layer = prepare_xor_learning()
    layer.train()
        
    fun(dataset, layer)
    
    layer.test()
    
    for sample in dataset.samples:
        loss = layer(sample)
        print(f"Output: {loss.output()}, Target: {loss.target()}, Loss: {loss.value()}")


In [50]:
print("Classic learning")
evaluate_testing(classic_learning)

Classic learning
Output: 0.0, Target: 0.0, Loss: 0.0
Output: 0.7574968842735786, Target: 1.0, Loss: 0.05880776113702211
Output: 0.7426854473631991, Target: 1.0, Loss: 0.06621077899867699
Output: 0.05052201839625794, Target: 0.0, Loss: 0.0025524743428318253


In [51]:
print("Classic learning (auto_backprop=True)")
evaluate_testing(classic_auto_backprop_learning)

Classic learning (auto_backprop=True)
Output: 0.0, Target: 0.0, Loss: 0.0
Output: 0.7574968842735786, Target: 1.0, Loss: 0.05880776113702211
Output: 0.7426854473631991, Target: 1.0, Loss: 0.06621077899867699
Output: 0.05052201839625794, Target: 0.0, Loss: 0.0025524743428318253


In [52]:
print("Batched learning")
evaluate_testing(sample_batching_learning)

Batched learning
Output: 0.0, Target: 0.0, Loss: 0.0
Output: 0.7574968842735786, Target: 1.0, Loss: 0.05880776113702211
Output: 0.7426854473631991, Target: 1.0, Loss: 0.06621077899867699
Output: 0.05052201839625794, Target: 0.0, Loss: 0.0025524743428318253


In [53]:
print("Batched learning (epochs=...)")
evaluate_testing(sample_batching_learning_delegate_epochs)

Batched learning (epochs=...)
Output: 0.0, Target: 0.0, Loss: 0.0
Output: 0.7574968842735786, Target: 1.0, Loss: 0.05880776113702211
Output: 0.7426854473631991, Target: 1.0, Loss: 0.06621077899867699
Output: 0.05052201839625794, Target: 0.0, Loss: 0.0025524743428318253


In [54]:
print("Batched learning (set_training_samples)")
evaluate_testing(sample_batching_learning_set_training_samples)

Batched learning (set_training_samples)
Output: 0.0, Target: 0.0, Loss: 0.0
Output: 0.7574968842735786, Target: 1.0, Loss: 0.05880776113702211
Output: 0.7426854473631991, Target: 1.0, Loss: 0.06621077899867699
Output: 0.05052201839625794, Target: 0.0, Loss: 0.0025524743428318253
